In [4]:
pip install ifcopenshell

In [32]:
import ifcopenshell
from ifcopenshell.util import element, classification, placement
import pandas as pd

# model = ifcopenshell.open("BHRDT.ifc")
# model = ifcopenshell.open("VRM.ifc")
# model = ifcopenshell.open("18I085-X-CVP-XX-XX-M3-X-9002-Showroom Typical Details_4.ifc")
model = ifcopenshell.open("18I085-X-CVP-XX-XX-M3-X-9002-Showroom Typical Details_2x3.ifc")
types = []
dict = {}
for storey in model.by_type("IfcBuildingStorey"):
    elements = element.get_decomposition(storey)

    for elem in elements:
        pset= element.get_psets(elem)
        subDict = {}
        ifcType = elem.is_a()
        subDict["ifcType"] = ifcType   
        classificationReference = list(classification.get_references(elem))
        for cl in classificationReference:   
            className = cl.ReferencedSource.Name + "_" +cl.ItemReference[:2]
            subDict[className + ": ItemReference"] = cl.ItemReference
            subDict[className + ": Name"] = cl.Name 
        if elem.Name:
            elementIdentity = elem.Name.split(':')
            if len(elementIdentity)>1:
                subDict['ElementName'] = elementIdentity[1]
        if pset:             
            for ps in pset.values():
                subDict.update(ps)                   
        subDict['ExtendedElementName'] = elementIdentity
        associations = elem.HasAssociations
        materialNumber = 0
        thkNumber = 0
        usedMaterials = []
        for i in associations:
            try:
                if i.RelatingMaterial.is_a('IfcMaterial'):
                    if i.RelatingMaterial.Name not in usedMaterials:   
                        usedMaterials.append(i.RelatingMaterial.Name)
                        materialNumber += 1
                        subDict["Material_" + str(materialNumber).zfill(3)] = i.RelatingMaterial.Name
                if i.RelatingMaterial.is_a('IfcMaterialList'):
                    for material in i.RelatingMaterial.Materials:
                        if material.Name not in usedMaterials:
                            usedMaterials.append(material.Name)
                            materialNumber += 1
                            subDict["Material_" + str(materialNumber).zfill(3)] = material.Name
                if i.RelatingMaterial.is_a('IfcMaterialLayerSetUsage'):
                    mLayers = list(i.RelatingMaterial.ForLayerSet.MaterialLayers)
                    for m in mLayers:
                        materialNumber += 1
                        subDict["Material_" + str(materialNumber).zfill(3)] = m.Material.Name
                        thkNumber += 1
                        subDict["LayerThk_" + str(materialNumber).zfill(3)] = m.LayerThickness
            except:
                pass
        locMatrix = placement.get_local_placement(elem.ObjectPlacement)
        x,y,z = locMatrix[0][-1], locMatrix[1][-1], locMatrix[2][-1]
        subDict["Global X"], subDict["Global Y"], subDict["Global Z"] = x,y,z
        #subDict['LocationMatrix'] = locMatrix
        dict[elem.GlobalId] = subDict

df = pd.DataFrame(dict)
df = df.transpose()

# matColumns = ["ifcType"] + [col for col in df.columns if "LayerThk_" in col] +[col for col in df.columns if "Material_" in col]
# matColumns = ["ifcType"] + [col for col in df.columns if "Material_" in col]
# df[df.ifcType == "IfcSlab"][matColumns].head(50)
#df.dropna(subset=["Material_051"])[matColumns].head(50)
# df[matColumns].head(50)

df


,ifcType,ElementName,Edited by,Assembly Code,Assembly Description,Type Name,Workset,id,Subcategory,Category,...,Width on side 2,Span,NetSurfaceArea,Categorize as,Location Line Offset,Angle Report,Arch_Lenght,DI_Radius,Offset Mullion,Mulion Offset
0OjgB$YifAduDyYAaDejEK,IfcBuildingElementProxy,Model Text 2400,,,,Model Text 2400,01_ARC_SHW Typical Details,1368,Generic Models,Generic Models,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0OjgB$YifAduDyYAaDejAe,IfcBuildingElementProxy,Model Text 1200,,,,Model Text 1200,01_ARC_SHW Typical Details,2420,Generic Models,Generic Models,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0OjgB$YifAduDyYAaDejAb,IfcBuildingElementProxy,Model Text 1200,,,,Model Text 1200,01_ARC_SHW Typical Details,4527,Generic Models,Generic Models,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0OjgB$YifAduDyYAaDejAZ,IfcBuildingElementProxy,Model Text 1200,,,,Model Text 1200,01_ARC_SHW Typical Details,5537,Generic Models,Generic Models,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0OjgB$YifAduDyYAaDejAy,IfcBuildingElementProxy,Model Text 1200,,,,Model Text 1200,01_ARC_SHW Typical Details,7323,Generic Models,Generic Models,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2wCRfN_fL6yweW04oz5_mQ,IfcMember,CM_CE_Aluminum Slats 200 x 30_-100,,,,CM_CE_Aluminum Slats 200 x 30_-100,01_ARC_SHW Typical Details,60913,NaN,Curtain Wall Mullions,...,15.0,2000.0,0.932,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2wCRfN_fL6yweW04oz5_mL,IfcMember,CM_CE_Aluminum Slats 200 x 30_-100,,,,CM_CE_Aluminum Slats 200 x 30_-100,01_ARC_SHW Typical Details,60922,NaN,Curtain Wall Mullions,...,15.0,2000.0,0.932,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2wCRfN_fL6yweW04oz5_mK,IfcMember,CM_CE_Aluminum Slats 200 x 30_-100,,,,CM_CE_Aluminum Slats 200 x 30_-100,01_ARC_SHW Typical Details,60931,NaN,Curtain Wall Mullions,...,15.0,2000.0,0.932,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2wCRfN_fL6yweW04oz5_mN,IfcMember,CM_CE_Aluminum Slats 200 x 30_-100,,,,CM_CE_Aluminum Slats 200 x 30_-100,01_ARC_SHW Typical Details,60940,NaN,Curtain Wall Mullions,...,15.0,2000.0,0.932,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# import ifcopenshell
# import pandas as pd

# # Load the IFC file
# ifc_file = ifcopenshell.open('18I085-X-CVP-XX-XX-M3-A-1001-Architecture.ifc')

# db = []
# # Get all the objects in the IFC file
# objects = ifc_file.by_type('IfcObject')
# for obj in objects:
#     pset= ifcopenshell.util.element.get_psets(obj)
#     db.append(obj.Name)
#     if pset:        
#         db.append(pset)
# db

# objects[2]
df[df.ID_brand][ID_brand]

Index(['ifcType', 'ElementName', 'Edited by', 'Assembly Code',
       'Assembly Description', 'Type Name', 'Workset', 'id', 'Subcategory',
       'Category',
       ...
       'Width on side 2', 'Span', 'NetSurfaceArea', 'Categorize as',
       'Location Line Offset', 'Angle Report', 'Arch_Lenght', 'DI_Radius',
       'Offset Mullion', 'Mulion Offset'],
      dtype='object', length=201)

In [8]:
df.dropna(subset = ['ID_Tag Code'])['ID_Tag Code']

1rD1yu5ar0$wrbg65LxK8u    FL_LF01
1MqurRZFb81O6JZ$MaCb5Z           
1MqurRZFb81O6JZ$MaCb40           
1MqurRZFb81O6JZ$MaCb4X           
1Z7BPZ6f505vq4Y$89LFDa    WA_IW01
                           ...   
3ntHXS08T6OQbJsbB8mSrm    CP_GP04
3ntHXS08T6OQbJsbB8mSrr    CP_GP05
3ntHXS08T6OQbJsbB8mSrD    CP_GP12
3ntHXS08T6OQbJsbB8mSrO    CP_GP06
3ntHXS08T6OQbJsbB8mSrG    CP_SP01
Name: ID_Tag Code, Length: 325, dtype: object

In [9]:
df["ifcType"]

0OjgB$YifAduDyYAaDejEK    IfcBuildingElementProxy
0OjgB$YifAduDyYAaDejAe    IfcBuildingElementProxy
0OjgB$YifAduDyYAaDejAb    IfcBuildingElementProxy
0OjgB$YifAduDyYAaDejAZ    IfcBuildingElementProxy
0OjgB$YifAduDyYAaDejAy    IfcBuildingElementProxy
                                   ...           
2wCRfN_fL6yweW04oz5_mQ                  IfcMember
2wCRfN_fL6yweW04oz5_mL                  IfcMember
2wCRfN_fL6yweW04oz5_mK                  IfcMember
2wCRfN_fL6yweW04oz5_mN                  IfcMember
2wCRfN_fL6yweW04oz5_py                  IfcMember
Name: ifcType, Length: 509, dtype: object